In [1]:
import numpy as np
import xarray as xr
from affine import Affine

import rioxarray
from rioxarray import Convention

# Create sample data
data = np.random.rand(100, 100)
da = xr.DataArray(
    data,
    dims=["y", "x"],
    coords={
        "x": np.linspace(-180, 180, 100),
        "y": np.linspace(-90, 90, 100)
    }
)

transform = Affine(3.6, 0.0, -180.0, 0.0, -1.8, 90.0)
print("Sample data created")

Sample data created


## CF Convention

The CF convention stores geospatial metadata in grid_mapping coordinate variables.

In [2]:
# Write CRS and transform using CF convention
da_cf = da.rio.write_crs("EPSG:4326", convention=Convention.CF)
da_cf = da_cf.rio.write_transform(transform, convention=Convention.CF)

print("CF Convention attributes:")
print(f"Grid mapping: {da_cf.attrs.get('grid_mapping')}")
print(f"Grid mapping coordinate: {list(da_cf.coords.keys())}")
print(f"Grid mapping attrs: {da_cf.coords['spatial_ref'].attrs.keys()}")
print(f"GeoTransform: {da_cf.coords['spatial_ref'].attrs.get('GeoTransform')}")

CF Convention attributes:
Grid mapping: spatial_ref
Grid mapping coordinate: ['x', 'y', 'spatial_ref']
Grid mapping attrs: dict_keys(['spatial_ref', 'crs_wkt', 'semi_major_axis', 'semi_minor_axis', 'inverse_flattening', 'reference_ellipsoid_name', 'longitude_of_prime_meridian', 'prime_meridian_name', 'geographic_crs_name', 'horizontal_datum_name', 'grid_mapping_name', 'GeoTransform'])
GeoTransform: 3.6 0.0 -180.0 0.0 -1.8 90.0


## Zarr Conventions

The Zarr conventions store geospatial metadata as direct attributes on the data array.

In [3]:
# Write CRS and transform using Zarr conventions
da_zarr = da.rio.write_crs("EPSG:4326", convention=Convention.Zarr)
da_zarr = da_zarr.rio.write_transform(transform, convention=Convention.Zarr)

print("Zarr Convention attributes:")
print(f"proj:code: {da_zarr.attrs.get('proj:code')}")
print(f"spatial:transform: {da_zarr.attrs.get('spatial:transform')}")
print(f"zarr_conventions: {[c['name'] for c in da_zarr.attrs.get('zarr_conventions', [])]}")

Zarr Convention attributes:
proj:code: None
spatial:transform: [3.6, 0.0, -180.0, 0.0, -1.8, 90.0]
zarr_conventions: ['proj:', 'spatial:']


## Zarr-Specific Methods

rioxarray provides specialized methods for working with Zarr conventions.

In [4]:
# Write CRS in multiple Zarr formats using convention module
from rioxarray._convention import zarr as zarr_conv

da_zarr_full = da.rio.write_crs("EPSG:4326", convention=Convention.Zarr)
da_zarr_full = zarr_conv.write_crs(da_zarr_full, da_zarr_full.rio.crs, format="all")

print("Multiple CRS formats:")
print(f"proj:code: {da_zarr_full.attrs.get('proj:code')}")
print(f"proj:wkt2: {da_zarr_full.attrs.get('proj:wkt2')[:50]}...")
print(f"proj:projjson type: {type(da_zarr_full.attrs.get('proj:projjson'))}")

Multiple CRS formats:
proj:code: EPSG:4326
proj:wkt2: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84"...
proj:projjson type: <class 'dict'>


In [5]:
# Write complete spatial metadata using convention module
da_spatial = da.rio.write_transform(transform, convention=Convention.Zarr)
da_spatial = zarr_conv.write_spatial_metadata(da_spatial, "y", "x", transform=transform)

print("Complete spatial metadata:")
print(f"spatial:dimensions: {da_spatial.attrs.get('spatial:dimensions')}")
print(f"spatial:shape: {da_spatial.attrs.get('spatial:shape')}")
print(f"spatial:bbox: {da_spatial.attrs.get('spatial:bbox')}")
print(f"spatial:registration: {da_spatial.attrs.get('spatial:registration')}")

Complete spatial metadata:
spatial:dimensions: ['y', 'x']
spatial:shape: [100, 100]
spatial:bbox: [-180.0, -90.0, 180.0, 90.0]
spatial:registration: pixel


In [6]:
# Write CRS and transform together
da_complete = da.rio.write_crs("EPSG:4326", convention=Convention.Zarr)
da_complete = da_complete.rio.write_transform(transform, convention=Convention.Zarr)

print("Complete Zarr conventions:")
print(f"Has CRS: {'proj:wkt2' in da_complete.attrs}")
print(f"Has transform: {'spatial:transform' in da_complete.attrs}")
print(f"Has dimensions: {'spatial:dimensions' in da_complete.attrs}")
print(f"Number of attributes: {len(da_complete.attrs)}")

Complete Zarr conventions:
Has CRS: True
Has transform: True
Has dimensions: False
Number of attributes: 3


## Global Convention Setting

You can set the default convention globally to avoid specifying it for each method call.

In [7]:
# Set Zarr as the global default
with rioxarray.set_options(convention=Convention.Zarr):
    da_global = da.rio.write_crs("EPSG:4326")  # Uses Zarr convention
    da_global = da_global.rio.write_transform(transform)  # Uses Zarr convention
    
    print("Using global Zarr convention:")
    print(f"proj:wkt2: {da_global.attrs.get('proj:wkt2', 'Not found')[:50] if da_global.attrs.get('proj:wkt2') else 'Not found'}...")
    print(f"spatial:transform: {da_global.attrs.get('spatial:transform')}")
    print(f"Has grid_mapping: {'grid_mapping' in da_global.attrs}")

Using global Zarr convention:
proj:wkt2: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84"...
spatial:transform: [3.6, 0.0, -180.0, 0.0, -1.8, 90.0]
Has grid_mapping: False


## Reading with Different Conventions

The reading behavior follows this priority: CF first (default), Zarr as fallback when explicitly declared.

In [8]:
# Create data with both conventions
da_both = da.rio.write_crs("EPSG:4326", convention=Convention.CF)
da_both = da_both.rio.write_crs("EPSG:4326", convention=Convention.Zarr)

print("Data with both conventions:")
print(f"Has CF grid_mapping: {'grid_mapping' in da_both.attrs}")
print(f"Has Zarr proj:code: {'proj:wkt2' in da_both.attrs}")

# Default reading (CF first, Zarr fallback)
crs_default = da_both.rio.crs
print(f"\nDefault reading (CF first): {crs_default}")

# Read using CF convention exclusively
with rioxarray.set_options(convention=Convention.CF):
    crs_cf = da_both.rio.crs
    print(f"CF convention only: {crs_cf}")

# Read using Zarr convention exclusively  
with rioxarray.set_options(convention=Convention.Zarr):
    crs_zarr = da_both.rio.crs
    print(f"Zarr convention only: {crs_zarr}")

Data with both conventions:
Has CF grid_mapping: True
Has Zarr proj:code: True

Default reading (CF first): EPSG:4326
CF convention only: EPSG:4326
Zarr convention only: EPSG:4326
